In [ ]:
import json
import difflib

from matplotlib import pyplot as plt
import matplotlib.cm as cm
from tqdm import tqdm

from network import *
from analyze_graph import *
import smopy
import pandas as pd


In [ ]:
network = Network()
network.set_stops("network.db")
stops_data = pd.read_csv("stops.csv", index_col="DHID")



In [ ]:
darmstadt=find_closest_station_id_by_name("Darmstadt Schloss",stops_data)


In [ ]:
stops_data.loc[darmstadt]

In [ ]:
start_station = darmstadt
time_limit = 30
start_time = time_to_minutes("09:00")
reachable = network.get_reachable_stations_in_time(start_station, start_time, time_limit)

In [ ]:
min_lat = 180
min_long = 180
max_lat = -180
max_long = -180

coordinates = []
for station in reachable:
    lat = stops_data.loc[station]["Latitude"]
    lon = stops_data.loc[station]["Longitude"]
    coordinates.append((lat, lon))

    min_lat = min(min_lat, lat)
    min_long = min(min_long, lon)
    max_lat = max(max_lat, lat)
    max_long = max(max_long, lon)

# area to plot
map_box = (min_lat, min_long, max_lat, max_long)
#print(map_box)
map = smopy.Map(map_box)

# figsize is used for resolution
ax = map.show_mpl(figsize=(12, 12))

# TODO annotation is not working correctly
ax.annotate(
    "Reachable from %s (%s) until %s (%i stops)" % (
        stops_data.loc[start_station, "Name"],
        minutes_to_time(start_time),
        minutes_to_time(start_time + time_limit),
        len(reachable)),
    xy=(0.5, 1.02),  # Position relative to axes (centered above the map)
    xycoords="axes fraction",
    fontsize=24,
    ha="center",  # Center horizontally
    va="bottom",  # Position below the top edge
)

for lat, lon in coordinates:
    x, y = map.to_pixels(lat, lon)
    ax.plot(x, y, 'or', ms=10, mew=2)

In [ ]:
map_box = (49.8388, 8.560719, 49.931479, 8.750582) # area around Darmstadt
# select stops in that area
in_area = stops_data[(stops_data["Latitude"] > map_box[0]) &
    (stops_data["Latitude"] < map_box[2]) &
    (stops_data["Longitude"] > map_box[1]) &
    (stops_data["Longitude"] < map_box[3]) ].copy()

In [ ]:
end_of_day = 3*60+50 #03:59
# a bus leaving at 0:30 is part of the old daythat is your ticket from the previous day is still valid

def get_early(stop_id):
    earliest_depature = 23*60+59 # 23:59
    for stop_id, timetable in network.stops[stop_id].items():
        timetable.sort()
        idx=0
        while idx < len(timetable) and timetable[idx][0] < end_of_day:
            idx += 1
        # found the next departure
        if idx < len(timetable):
            earliest_depature = min(timetable[idx][0],earliest_depature)
    return earliest_depature

# to_series as, as the stop_id is the index
in_area["first_departure"] = in_area.index.to_series().apply(get_early)

In [ ]:
from matplotlib.lines import Line2D

in_area["first_departure_hour"] =in_area["first_departure"] //60 # only the hour

unique_hours = sorted(in_area["first_departure_hour"].unique())
n_colors = len(unique_hours)
colors = cm.get_cmap("tab10", n_colors)  # Or "tab20", "Set3", etc.
hour_to_color = {hour: colors(i) for i, hour in enumerate(unique_hours)}

map = smopy.Map(map_box)

# figsize is used for resolution
ax = map.show_mpl(figsize=(12, 12))

# TODO annotation is not working correctly
ax.annotate(
    "Earliest Depature (after 03:50)",
    xy=(0.5, 1.02),  # Position relative to axes (centered above the map)
    xycoords="axes fraction",
    fontsize=24,
    ha="center",  # Center horizontally
    va="bottom",  # Position below the top edge
)

for _, row in in_area.iterrows():
    x, y = map.to_pixels(row["Latitude"], row["Longitude"])
    ax.plot(x, y, 'or',color=hour_to_color[row["first_departure_hour"]], ms=10, mew=2)

legend_elements = [
    Line2D([0], [0], marker='o', color='w', label=f"{hour}:00",
           markerfacecolor=color, markersize=10)
    for hour, color in hour_to_color.items() if hour != 23
]
ax.legend(handles=legend_elements, title="First Departure Hour")